<a href="https://colab.research.google.com/github/elijahmflomo/Advance-Data-Structure-and-Algorithm/blob/main/ADSA_LAB_TEST_multiple_flights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
Airline Ticket Reservation (multiple flights)
"""

from dataclasses import dataclass
from typing import Optional


# -- Passenger linked list (unchanged logic) ------
@dataclass
class PassengerNode:
    name: str
    next: Optional["PassengerNode"] = None


class PassengerList:
    """Alphabetized singly linked list of passenger names."""

    def __init__(self):
        self.head: Optional[PassengerNode] = None

    def is_empty(self) -> bool:
        return self.head is None

    def insert(self, name: str) -> bool:
        name = name.strip()
        if not name:
            raise ValueError("Name cannot be empty.")

        new_node = PassengerNode(name)
        if self.head is None or name.lower() < self.head.name.lower():
            if self.head and self.head.name.lower() == name.lower():
                return False
            new_node.next = self.head
            self.head = new_node
            return True

        prev = None
        current = self.head
        while current and current.name.lower() <= name.lower():
            if current.name.lower() == name.lower():
                return False
            prev = current
            current = current.next

        prev.next = new_node
        new_node.next = current
        return True

    def remove(self, name: str) -> bool:
        name = name.strip()
        if self.head is None:
            return False

        if self.head.name.lower() == name.lower():
            self.head = self.head.next
            return True

        prev = self.head
        current = self.head.next
        while current:
            if current.name.lower() == name.lower():
                prev.next = current.next
                return True
            prev = current
            current = current.next

        return False

    def search(self, name: str) -> bool:
        name = name.strip()
        current = self.head
        while current:
            if current.name.lower() == name.lower():
                return True
            if current.name.lower() > name.lower():
                return False
            current = current.next
        return False

    def to_list(self) -> list:
        out = []
        current = self.head
        while current:
            out.append(current.name)
            current = current.next
        return out

    def display(self) -> None:
        if self.head is None:
            print("  No passengers reserved.")
            return
        print("  Passengers (alphabetical):")
        idx = 1
        current = self.head
        while current:
            print(f"    {idx}. {current.name}")
            idx += 1
            current = current.next

In [4]:
# -- Flight linked list -----
@dataclass
class FlightNode:
    flight_no: str
    passengers: PassengerList
    next: Optional["FlightNode"] = None


class FlightList:
    """Singly linked list of flights. flight_no is treated case-insensitively for identity."""

    def __init__(self):
        self.head: Optional[FlightNode] = None

    def add_flight(self, flight_no: str) -> bool:
        flight_no = flight_no.strip()
        if not flight_no:
            raise ValueError("Flight number cannot be empty.")
        if self.find_flight(flight_no) is not None:
            return False  # already exists
        new_flight = FlightNode(flight_no, PassengerList())
        # Insert at head for simplicity (no ordering required for flights)
        new_flight.next = self.head
        self.head = new_flight
        return True

    def remove_flight(self, flight_no: str) -> bool:
        flight_no = flight_no.strip()
        if self.head is None:
            return False
        if self.head.flight_no.lower() == flight_no.lower():
            self.head = self.head.next
            return True
        prev = self.head
        current = self.head.next
        while current:
            if current.flight_no.lower() == flight_no.lower():
                prev.next = current.next
                return True
            prev = current
            current = current.next
        return False

    def find_flight(self, flight_no: str) -> Optional[FlightNode]:
        flight_no = flight_no.strip()
        current = self.head
        while current:
            if current.flight_no.lower() == flight_no.lower():
                return current
            current = current.next
        return None

    def list_flights(self) -> list:
        out = []
        current = self.head
        while current:
            out.append(current.flight_no)
            current = current.next
        return out

    def display_flights(self) -> None:
        flights = self.list_flights()
        if not flights:
            print("No flights available.")
            return
        print("Available flights:")
        idx = 1
        current = self.head
        while current:
            print(f"  {idx}. {current.flight_no}  (passengers: {len(current.passengers.to_list())})")
            idx += 1
            current = current.next

In [5]:
# ---CLI / Menu ---
def flight_submenu(flight: FlightNode):
    """Passenger operations for a selected flight."""
    while True:
        print(f"\n--- Managing Flight: {flight.flight_no} ---")
        print("1. Reserve a ticket")
        print("2. Cancel a reservation")
        print("3. Check reservation for a person")
        print("4. Display passengers")
        print("0. Back to main menu")
        choice = input("Choose an option: ").strip()
        if choice == "1":
            name = input(" Enter passenger name to reserve: ").strip()
            if not name:
                print("  Name cannot be empty.")
                continue
            try:
                ok = flight.passengers.insert(name)
                print("  Ticket reserved." if ok else "  Reservation failed: already reserved.")
            except ValueError as e:
                print("  Error:", e)

        elif choice == "2":
            name = input(" Enter passenger name to cancel: ").strip()
            if not name:
                print("  Name cannot be empty.")
                continue
            ok = flight.passengers.remove(name)
            print("  Reservation cancelled." if ok else "  No reservation found for that name.")

        elif choice == "3":
            name = input(" Enter name to check: ").strip()
            if not name:
                print("  Name cannot be empty.")
                continue
            found = flight.passengers.search(name)
            print(f"  {name} has a reservation." if found else f"  {name} does NOT have a reservation.")

        elif choice == "4":
            flight.passengers.display()

        elif choice == "0":
            break
        else:
            print("  Invalid choice. Enter 0-4.")

In [ ]:
def demo_multi(flist: FlightList):
    """Demo to create multiple flights with passengers and show operations."""
    print("\n--- Demo: creating sample flights and reservations ---")
    # Reset flight list
    flist.head = None
    # Create flights
    sample_flights = ["AI101", "BF202", "CX303"]
    for f in sample_flights:
        flist.add_flight(f)
        print(f"Added flight: {f}")

    # Add passengers to AI101
    f = flist.find_flight("AI101")
    if f:
        for name in ["Alice", "bob smith", "adam west", "Charlie"]:
            inserted = f.passengers.insert(name)
            print(f"AI101: Insert '{name}':", "OK" if inserted else "Already exists")

    # Add passengers to BF202
    f2 = flist.find_flight("BF202")
    if f2:
        for name in ["Zara", "Yusuf", "Maya"]:
            f2.passengers.insert(name)

    print("\nCurrent flights and passengers:")
    flist.display_flights()
    print("\nPassengers on AI101:")
    flist.find_flight("AI101").passengers.display()
    print("--- End demo ---\n")


def main_menu_multi():
    flight_list = FlightList()
    menu = """\n=== Airline Reservation (Multiple Flights) ===
1. Add a flight
2. Remove a flight
3. List flights
4. Manage a flight (passengers)
5. Demo / run sample actions
0. Exit
Choose an option: """
    while True:
        try:
            choice = input(menu).strip()
            if choice == "1":
                flight_no = input("Enter new flight number (e.g., AI101): ").strip()
                if not flight_no:
                    print("Flight number cannot be empty.")
                    continue
                ok = flight_list.add_flight(flight_no)
                print("Flight added." if ok else "Flight already exists.")

            elif choice == "2":
                flight_no = input("Enter flight number to remove: ").strip()
                if not flight_no:
                    print("Flight number cannot be empty.")
                    continue
                ok = flight_list.remove_flight(flight_no)
                print("Flight removed." if ok else "No such flight.")

            elif choice == "3":
                flight_list.display_flights()

            elif choice == "4":
                flight_no = input("Enter flight number to manage: ").strip()
                if not flight_no:
                    print("Flight number cannot be empty.")
                    continue
                flight = flight_list.find_flight(flight_no)
                if not flight:
                    print("No such flight. You can add it from main menu.")
                    continue
                flight_submenu(flight)

            elif choice == "5":
                demo_multi(flight_list)

            elif choice == "0":
                print("Exiting. Goodbye.")
                break
            else:
                print("Invalid choice. Enter 0-5.")
        except ValueError as ve:
            print("Input error:", ve)
        except KeyboardInterrupt:
            print("\nInterrupted. Exiting.")
            break


if __name__ == "__main__":
    main_menu_multi()



=== Airline Reservation (Multiple Flights) ===
1. Add a flight
2. Remove a flight
3. List flights
4. Manage a flight (passengers)
5. Demo / run sample actions
0. Exit
Choose an option: 1
Enter new flight number (e.g., AI101):  AI101
Flight added.

=== Airline Reservation (Multiple Flights) ===
1. Add a flight
2. Remove a flight
3. List flights
4. Manage a flight (passengers)
5. Demo / run sample actions
0. Exit
Choose an option: 4
Enter flight number to manage:  AI101

--- Managing Flight: AI101 ---
1. Reserve a ticket
2. Cancel a reservation
3. Check reservation for a person
4. Display passengers
0. Back to main menu
Choose an option: 4
  No passengers reserved.

--- Managing Flight: AI101 ---
1. Reserve a ticket
2. Cancel a reservation
3. Check reservation for a person
4. Display passengers
0. Back to main menu
Choose an option: 3
 Enter name to check: Elijah
  Elijah does NOT have a reservation.

--- Managing Flight: AI101 ---
1. Reserve a ticket
2. Cancel a reservation
3. Check re